### 한글 폰트 설치

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

형태소 분석을 위한 환경 설정

In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev  # 자바머신 설치
!pip3 install JPype1-py3    
!pip3 install konlpy
!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"   # 환경 설정

In [ ]:
from konlpy.tag import Okt

okt = Okt()
print(okt.morphs("아버지가방에들어가신다"))

Mecab 사용시 설치 코드

In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996 


In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()
print(mecab.morphs("아버지가방에들어가신다"))

### [연습] 2022년 대통령 신년사 형태소 분석

In [ ]:
from konlpy.tag import Okt
from konlpy.tag import Mecab
from nltk import Text
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk

import warnings
warnings.simplefilter('ignore')

1. 파일 읽어오기

In [ ]:
f = open("./2022_newyear.txt")
texts = f.read()
f.close()

In [ ]:
texts[:50]

2. 읽어온 데이터를 이용해 형태소 단위로 분류하기
- 명사/동사/형용사 추출

In [ ]:
okt = Okt()
okt.morphs(texts)   # 단어 단위로 분류하기

pos_txt=okt.pos(texts) # 품사 단위로 분류

Nouns =[ t[0] for t in pos_txt if t[1] == "Noun" ]
Adjs =[ t[0] for t in pos_txt if t[1] == "Adjective" ]
Verbs =[ t[0] for t in pos_txt if t[1] == "Verb" ]

Nouns[:5]
Adjs[:5]
Verbs[:5]

In [ ]:
mecab = Mecab()
mecab.morphs(texts)   # 단어 단위로 분류하기

pos_txt=mecab.pos(texts) # 품사 단위로 분류

Nouns =[ t[0] for t in pos_txt if t[1] == "Noun" ]
Adjs =[ t[0] for t in pos_txt if t[1] == "Adjective" ]
Verbs =[ t[0] for t in pos_txt if t[1] == "Verb" ]

Nouns[:5]
Adjs[:5]
Verbs[:5]

3. 분류한 데이터 하나로 합친후 Text를 이용해 NLTK.text로 만들기

In [ ]:
morph = Nouns + Adjs + Verbs
print(morph[:10])
morph_T = Text(morph)
morph_T

4. 토큰(단어) 빈도수 그래프로 출력하기

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 

In [ ]:
plt.figure(figsize=(14, 5))
plt.title("2022년 대통령 신년사", size=15)
morph_T.plot()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
morph_T.dispersion_plot(["국민", '경제', "코로나"])   # 단어 위치 확인
plt.show()

## 빈도수 계산 및 워드클라우드 작성

#### 불용어(Stepword) 처리
- 영문 사전: 영문은 기본 제공
- 한글 사전: 개인적을 만들어야 함
- 사전 참조: https://www.ranks.nl/stopwords

In [ ]:
# 사용자 stop_words 만들기(한글인 경우 꼭 필요)
stop_words = ". 섰습니다 맞습니다 있습니다 하겠습니다 수 할 위 해 등 ? 출발 선 다시 입니다"
stop_words = stop_words.split(" ")
print(stop_words)

In [ ]:
# 사용자 stopword 적용하기
# morph_sw = []
# for sw in morph:
#     if sw not in stop_words:
#         morph_sw.append(sw)

morph_sw = [ sw for sw in morph if sw not in stop_words]

print(morph[:10])
print(morph_sw[:10])

### 빈도수에 의한 워드클리우드 출력


In [ ]:
morph_cnt = Counter(morph_sw)   # 리트에 입력된 데이터의 빈도수 계산
morph_cnt  # 딕셔너리 구조: {단어:빈도수, 단어:빈도수, ....}

In [ ]:
morph_txt = morph_cnt.most_common()   # 딕셔너리형 자료를 튜플구조로 만들어 리스트형으로 제공
morph_txt[:10]

In [ ]:
## 빈도수가 낮은 데이터(빈도수 5 미만인 데이터) 제거
morph_txt_up = [tc for tc in morph_txt if tc[1] >= 5]

# 워드클라우드 작업을 위한 딕셔너리(튜플 = > 딕셔너리) 구조 변경
morph_txt_up = dict(morph_txt_up)

## 워드클라우드 출력
#font_path = './'
wordcloud = WordCloud(font_path = "NanumBarunGothic", 
                      background_color='white', 
                      width=1000, height=1000,
                      max_words=100, max_font_size=300)

# 틱셔너리 구조의 데이터를 이용해 워크클라우드 출력
wc = wordcloud.generate_from_frequencies(morph_txt_up)
plt.figure(figsize=(8, 15))
plt.imshow(wc)
plt.imshow(wc, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
import pandas as pd

# 빈도수가 5 이상인 데이터를 이용해 DataFrame로 출력

morph_txt_tu=[tc for tc in morph_txt if tc[1] >= 5]
df = pd.DataFrame(morph_txt_tu, columns=['단어', '빈도수'])
df